In [31]:
import pandapower as pp
import pandas as pd
import numpy as np
from pandapower.timeseries.output_writer import OutputWriter


# Install pandapower in a new environment by pip install pandapower==2.13.1
net = pp.from_pickle('./aggregated_grid_2018_with_generators_loads_costs_controllers.p')

out_path = "./AC_OPF/Results Final OPF"
ow = OutputWriter(net, output_path=out_path, output_file_type=".csv", csv_separator=",")
# ow.remove_log_variable("res_gen", "vm_pu")

In [32]:
# TIMESTEP-WISE SIMULATION TO GET SIMULATED VALUES

# Simulate for a single timestep (t) using function run (pp.runpp, pp.runopp,...) and give the outputs at net.res_...
def get_simulated_values(t, run):
    for controller in net.controller.object:
        controller.time_step(net, t)

    run(net, verbose=True)

In [33]:
# GET FROM SIMULATION

def get_from_simulation(res_csv, component_numbers):
    # Join simulation timeseries dataframes in a dataframe
    bus_timesteps = pd.concat([res_csv[f"{g}"] for g in component_numbers], axis=1)

    return bus_timesteps

In [34]:
# GET FROM CONTROLLER

def get_from_controller(start_controller_number, component_numbers):
    # Join controller timeseries dataframes at the same bus in a dataframe
    bus_timesteps = pd.concat([net.controller.object[start_controller_number + g].data_source.to_dict()['df'].iloc[:,0] for g in component_numbers], axis=1)

    return bus_timesteps

In [35]:
# GET AGGREGATED DATA

total_bus_numbers = net.bus.index

def get_aggregated_x(res_x, bus_gens, get_from):
    # res_x = csv for get_from_csv or start controller number for get_from_controller
    gen_busses_x = [] # List of timestep-wise gen x dataframes for each bus

    for bus in bus_gens:
        # Join timeseries dataframes (q_mvar, vm_pu and va_degree) at the same bus in a dataframe
        bus_timesteps = get_from(res_x, bus)

        # Add timestep-wise values together
        bus_timesteps_total = bus_timesteps.sum(axis=1)
        
        gen_busses_x.append(bus_timesteps_total)

    gen_x = pd.concat(gen_busses_x, axis=1) # Put busses together in one df (rows = timesteps, columns = buses)

    gen_x.columns = bus_gens.index # Column names to bus numbers

    gen_x = gen_x.reindex(range(total_bus_numbers[0], total_bus_numbers[-1] + 1), axis=1, fill_value=0) # Fill missing bus numbers with q_mvar = 0
    
    return gen_x



# gen_busses_q_mvar = [] 
# gen_busses_vm_pu = []
# gen_busses_va_degree = []



# for bus in bus_gens:
#     # Join timeseries dataframes (q_mvar, vm_pu and va_degree) at the same bus in a dataframe
#     bus_timesteps = pd.concat([res_q_mvar[f"{g}"] for g in bus], axis=1)
#     bus_timesteps_vm = pd.concat([res_vm_pu[f"{vm}"] for vm in bus], axis=1)
#     bus_timesteps_va = pd.concat([res_va_degree[f"{va}"] for va in bus], axis=1)

#     # Add timestep-wise values together
#     bus_timesteps_total = bus_timesteps.sum(axis=1)
#     bus_timesteps_total_vm = bus_timesteps_vm.sum(axis=1)
#     bus_timesteps_total_va = bus_timesteps_va.sum(axis=1)

#     gen_busses_x.append(bus_timesteps_total)
#     gen_busses_vm_pu.append(bus_timesteps_total_vm)    
#     gen_busses_va_degree.append(bus_timesteps_total_va)

# gen_q_mvar = pd.concat(gen_busses_x, axis=1) # Put busses together in one df (rows = timesteps, columns = buses)
# gen_vm_pu = pd.concat(gen_busses_vm_pu, axis=1) # Put busses together in one df (rows = timesteps, columns = buses)
# gen_va_degree = pd.concat(gen_busses_va_degree, axis=1) # Put busses together in one df (rows = timesteps, columns = buses)

# gen_q_mvar.columns = bus_gens.index # Column names to bus numbers
# gen_vm_pu.columns = bus_gens.index # Column names to bus numbers
# gen_va_degree.columns = bus_gens.index # Column names to bus numbers

# gen_q_mvar = gen_q_mvar.reindex(range(total_bus_numbers[0], total_bus_numbers[-1] + 1), axis=1, fill_value=0) # Fill missing bus numbers with q_mvar = 0
# gen_vm_pu = gen_vm_pu.reindex(range(total_bus_numbers[0], total_bus_numbers[-1] + 1), axis=1, fill_value=0) # Fill missing bus numbers with q_mvar = 0
# gen_va_degree = gen_va_degree.reindex(range(total_bus_numbers[0], total_bus_numbers[-1] + 1), axis=1, fill_value=0) # Fill missing bus numbers with q_mvar = 0

# gen_vm_pu.head(100)    


In [36]:
# CREATE LOAD Q_MVAR

pf = 0.95 # Using a power factor 0.95
p_to_q = np.sin(np.arccos(pf))/pf # P to Q constant

loads_p_mw = get_from_controller(0, net.load.index) # All controller p_mw data

loads_q_mvar = loads_p_mw.mul(p_to_q)

# loads_q_mvar.head(50)


In [37]:
# CREATE LOAD Q_MVAR CONTROLLER

for l in range(len(net.load.index)):
    controller_load_q_mvar = loads_p_mw.iloc[:, l].multiply(p_to_q)
    dataset = controller_load_q_mvar.to_frame()
    prof_name = dataset.columns[0]
    dataset = pp.timeseries.data_sources.frame_data.DFData(dataset)
    
    pp.control.controller.const_control.ConstControl(net, "load", "q_mvar", element_index=l, 
                                                 data_source=dataset, 
                                                 recycle={'trafo': False, 'gen': False, 'bus_pq': True}, 
                                                 profile_name=prof_name)

# net.controller

In [38]:
# SET REASONABLE Q_MVAR BOUNDARIES FOR GEN

for i in net.gen.index:
    net.gen.loc[i, "min_q_mvar"] = -1000.0
    net.gen.loc[i, "max_q_mvar"] = 1000.0

In [39]:
# SET REASONABLE Q_MVAR BOUNDARIES FOR EXT_GRID

for i in net.ext_grid.index:
    net.ext_grid.loc[i, "min_q_mvar"] = -5000.0
    net.ext_grid.loc[i, "max_q_mvar"] = 5000.0

net.ext_grid.loc[0, "type"] = "External Grid" # Fix ext_grid 0
net.ext_grid.loc[0, "p_mw"] = 0.0

# net.ext_grid

In [40]:
# FIX MAIN SLACK BUS (35) AS NON-CONTROLLABLE

# net.ext_grid.controllable[7] = False

In [41]:
# FIX MAIN SLACK BUS (35) AT 1.0 p.u. BY CHANGING MIN AND MAX v_pu

net.bus.loc[35, "min_vm_pu"] = 1.0
net.bus.loc[35, "max_vm_pu"] = 1.0

In [42]:
# LOG SIMULATION VARIABLES

ow.log_variable('res_bus', 'vm_pu') # Add logging for Bus
ow.log_variable('res_bus', 'va_degree') 
ow.log_variable('res_bus', 'p_mw') 
ow.log_variable('res_bus', 'q_mvar')

ow.log_variable('res_line', 'loading_percent') # Line
ow.log_variable('res_line', 'i_from_ka') 
ow.log_variable('res_line', 'i_to_ka') 
ow.log_variable('res_line', 'i_ka') 
ow.log_variable('res_line', 'p_from_mw')
ow.log_variable('res_line', 'q_from_mvar') 
ow.log_variable('res_line', 'p_to_mw') 
ow.log_variable('res_line', 'q_to_mvar') 
ow.log_variable('res_line', 'vm_from_pu') 
ow.log_variable('res_line', 'vm_to_pu') 
ow.log_variable('res_line', 'va_from_degree') 
ow.log_variable('res_line', 'va_to_degree') 

ow.log_variable('res_gen', 'p_mw') # Gen
ow.log_variable('res_gen', "q_mvar")
ow.log_variable('res_gen', "va_degree")
ow.log_variable('res_gen', "vm_pu")

ow.log_variable('res_sgen', 'p_mw') # Sgen
ow.log_variable('res_sgen', 'q_mvar')

ow.log_variable('res_load', 'p_mw') # Load
ow.log_variable('res_load', 'q_mvar')

ow.log_variable('res_ext_grid', 'p_mw') # Ext grid
ow.log_variable('res_ext_grid', 'q_mvar')

ow.log_variable('res_trafo', 'loading_percent') # Transformer
ow.log_variable('res_trafo', 'p_hv_mw')
ow.log_variable('res_trafo', 'p_hv_mvar')
ow.log_variable('res_trafo', 'p_lv_mw')
ow.log_variable('res_trafo', 'q_lv_mvar')
ow.log_variable('res_trafo', 'pl_mw')
ow.log_variable('res_trafo', 'ql_mvar')
ow.log_variable('res_trafo', 'i_hv_ka')
ow.log_variable('res_trafo', 'i_lv_ka')
ow.log_variable('res_trafo', 'vm_hv_pu')
ow.log_variable('res_trafo', 'va_hv_degree')
ow.log_variable('res_trafo', 'vm_lv_pu')
ow.log_variable('res_trafo', 'va_lv_degree')

In [43]:
# GET TIMESERIES SIMULATION RESULTS

# start_time = 0
# total_time = 8760
# pp.timeseries.run_timeseries(net, run=pp.runopp, continue_on_divergence=True) # Ignore divergence 

start_time = 6912
total_time = 72
pp.timeseries.run_timeseries(net, run=pp.runopp, continue_on_divergence=True, time_steps=range(start_time, start_time + total_time))

c:\Users\bjorn\miniconda3\lib\site-packages\pandapower\timeseries\output_writer.py:177: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'range(6912, 6984)' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  self.output["Parameters"].loc[:, "time_step"] = self.time_steps
c:\Users\bjorn\miniconda3\lib\site-packages\pandapower\control\run_control.py:50: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  level = controller.level.fillna(0).apply(asarray).values
  1%|▏         | 1/72 [00:00<00:31,  2.28it/s]c:\Users\bjorn\miniconda3\lib\site-packages\pandapower\opf\validate_opf_input.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through c

In [168]:
# *OR* GET INDIVIDUAL SIMULATION STEP

get_simulated_values(t=6912, run=pp.runopp)

c:\Users\bjorn\miniconda3\lib\site-packages\pandapower\opf\validate_opf_input.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  net[element_type].controllable.fillna(True, inplace=True)
c:\Users\bjorn\miniconda3\lib\site-packages\pandapower\opf\validate_opf_input.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are sett

PYPOWER Version 5.1.4, 27-June-2018 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!

Converged in 0.82 seconds
Objective Function Value = 338193.11 $/hr
| PyPower (ppci) System Summary - these are not valid for pandapower DataFrames|

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses             45     Total Gen Capacity   29928.3       -105000.0 to 105000.0
Generators       103     On-line Capacity     29928.3       -105000.0 to 105000.0
Committed Gens   103     Generation (actual)   9175.7            2153.0
Loads             28     Load                  9152.0            3008.1
  Fixed           28       Fixed               9152.0            3008.1
  Dispatchable     0       Dispatchable           0.0 of 0.0        0.0
Shunts             0     Shunt (inj)              0.0               0.0
Branches          95     Losses (I^2 

c:\Users\bjorn\miniconda3\lib\site-packages\pandapower\results_bus.py:212: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  net[res_]["p_mw"].loc[controlled_elements] = ppc["gen"][gen_idx, PG] * gen_sign
c:\Users\bjorn\miniconda3\lib\site-packa

In [145]:
# SWAPPING SLACK BUS (35)

def swap_slack(df, slack_number): # Switch slack bus position

    copy = df.iloc[:, 0].copy()
    df.iloc[:, 0] = df.iloc[:, slack_number]
    df.iloc[:, slack_number] = copy
    
slack_bus = 35

In [146]:
# GET GENERATION P_MW

gen_active = net.gen.loc[net.gen["in_service"] == True] # Drop unused generators
bus_gens = gen_active.groupby("bus").apply(lambda x: x.index.tolist()) # Group generators by bus

res_gen_p_mw = pd.read_csv(out_path + "/res_gen/p_mw.csv") # Read from simulation result

gen_p_mw = get_aggregated_x(res_gen_p_mw, bus_gens, get_from_simulation) # Get bus-aggregated generation

swap_slack(gen_p_mw, slack_bus)

gen_p_mw.head(10)

C:\Users\bjorn\AppData\Local\Temp\ipykernel_30188\4047868020.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bus_gens = gen_active.groupby("bus").apply(lambda x: x.index.tolist()) # Group generators by bus


bus,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
0,0,0,107.363991,1.430079e-04,0,0,0.007415,0,50.002769,47.999972,462.348377,548.337351,533.076109,82.955682,125.427671,237.662899,24.999978,0,0.004656,0,0,1580.006751,1202.987553,82.997776,219.962072,256.488772,229.998539,0,1801.040480,1096.772492,56.983032,0,0,111.945481,0,0,0,0,0,0,0,0,0,0,0
1,0,0,111.254091,2.701192e-04,0,0,0.014339,0,50.005361,47.999947,446.843404,532.214150,527.961142,82.907449,87.796057,230.268226,24.999959,0,0.009014,0,0,1580.013069,1202.975823,82.995755,219.926810,268.874442,229.997203,0,1801.076323,1094.804069,56.966887,0,0,111.874662,0,0,0,0,0,0,0,0,0,0,0
2,0,0,114.786084,5.242111e-05,0,0,0.002723,0,50.001015,47.999990,434.324215,515.832530,524.098565,82.977268,49.722899,220.624677,24.999992,0,0.001715,0,0,1580.002488,1202.995118,82.999167,219.984286,286.588853,229.999450,0,1801.014105,1095.608432,56.993809,0,0,111.960865,0,0,0,0,0,0,0,0,0,0,0
3,0,0,123.329362,4.903530e-05,0,0,0.002581,0,50.000965,47.999990,434.551210,511.451579,524.047803,82.977626,43.168903,220.549457,24.999993,0,0.001624,0,0,1580.002356,1202.995494,82.999230,219.985502,314.778231,229.999489,0,1801.013137,1095.074501,56.994421,0,0,111.957943,0,0,0,0,0,0,0,0,0,0,0
4,0,0,127.787139,1.985096e-04,0,0,0.011373,0,50.004319,47.999961,456.023301,526.217365,526.360939,82.924910,97.613491,241.293929,24.999970,0,0.007129,0,0,1580.010332,1202.982840,82.997073,219.945874,371.470290,229.998018,0,1801.054510,1095.907493,56.977752,0,0,111.893681,0,0,0,0,0,0,0,0,0,0,0
5,0,0,127.996836,3.676046e-05,0,0,0.002906,0,50.001182,47.999993,571.999056,645.145347,561.989840,82.994441,490.519894,413.900174,24.999994,0,0.001792,0,0,1580.002583,1202.997694,82.999577,219.993918,648.460649,229.999697,0,1801.012623,1096.985628,56.996231,0,0,111.994564,0,0,0,0,0,0,0,0,0,0,0
6,0,0,127.999987,7.923809e-07,0,0,190.143520,0,229.999698,48.000000,575.999939,1122.022055,561.999982,82.999980,789.999971,800.999915,25.000000,0,0.000654,0,0,1580.000626,1202.999979,82.999995,219.999980,840.999962,229.999996,0,2898.048084,1096.999968,56.999957,0,0,111.999979,0,0,0,0,0,0,0,0,0,0,0
7,0,0,127.999872,1.691910e-05,0,0,1078.826560,0,229.998375,47.999997,575.999704,1809.999033,561.999901,82.999902,789.999892,800.999581,24.999998,0,319.394392,0,0,1580.035653,1202.999817,82.999933,219.999917,840.999757,229.999949,0,3399.999306,1096.999811,56.999709,0,0,111.999901,0,0,0,0,0,0,0,0,0,0,0
8,0,0,127.999987,2.038371e-06,0,0,1078.995881,0,229.999842,48.000000,575.999972,1809.999939,561.999991,82.999991,789.999990,800.999961,25.000000,0,652.508450,0,0,1580.004165,1202.999982,82.999993,219.999993,840.999976,229.999995,0,3399.999941,1096.999982,56.999975,0,0,111.999991,0,0,0,0,0,0,0,0,0,0,0
9,0,0,127.999991,1.372572e-06,0,0,1078.996837,0,229.999892,48.000000,575.999981,1809.999955,561.999994,82.999994,789.999993,800.999973,25.000000,0,567.585676,0,0,1580.002785,1202.999988,82.999995,219.999995,840.999984,229.999996,0,3399.999959,1096.999988,56.999983,0,0,111.999994,0,0,0,0,0,0,0,0,0,0,0


In [147]:
# GET BUS VM_PU

res_vm_pu = pd.read_csv(out_path + "/res_bus/vm_pu.csv")

vm_pu = res_vm_pu.drop(columns=["Unnamed: 0"])

swap_slack(vm_pu, slack_bus)

vm_pu.head(20)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
0,1.0,1.029049,1.031064,1.032532,1.035399,1.030620,1.038637,1.035664,1.037715,1.034367,1.014351,1.016606,1.014702,1.011553,1.010036,1.017661,1.017242,1.017199,1.039893,1.013366,1.025610,1.040005,1.014393,1.024204,1.012277,1.023365,1.014955,1.037434,1.000030,1.015458,1.015289,1.013789,1.006395,1.009631,1.026833,1.038655,1.040005,1.039893,1.037788,1.024204,1.020368,1.014955,1.014955,1.014955,1.015289
1,1.0,1.027921,1.029835,1.031199,1.033957,1.029329,1.037174,1.034292,1.036259,1.033027,1.013775,1.015972,1.014130,1.011101,1.009614,1.016966,1.016409,1.016366,1.038412,1.012757,1.024654,1.038521,1.013854,1.023191,1.011789,1.022467,1.014274,1.036054,1.000029,1.014861,1.014687,1.013242,1.006109,1.009255,1.025864,1.037192,1.038521,1.038412,1.036376,1.023191,1.019438,1.014274,1.014274,1.014274,1.014687
2,1.0,1.027608,1.029537,1.030825,1.033619,1.028919,1.036871,1.033969,1.035950,1.032729,1.013534,1.015656,1.013920,1.010927,1.009424,1.016696,1.016120,1.016104,1.038024,1.012525,1.024340,1.038133,1.013631,1.022840,1.011584,1.022144,1.014021,1.035687,1.000029,1.014640,1.014405,1.013039,1.006028,1.009118,1.025579,1.036887,1.038133,1.038024,1.035993,1.022840,1.019109,1.014021,1.014021,1.014021,1.014405
3,1.0,1.027595,1.029537,1.030713,1.033522,1.028807,1.036778,1.033880,1.035849,1.032654,1.013567,1.015716,1.013962,1.010967,1.009451,1.016728,1.016123,1.016086,1.037923,1.012554,1.024342,1.038031,1.013681,1.022826,1.011621,1.022177,1.014056,1.035601,1.000029,1.014675,1.014463,1.013080,1.006050,1.009152,1.025554,1.036795,1.038031,1.037923,1.035902,1.022826,1.019106,1.014056,1.014056,1.014056,1.014463
4,1.0,1.028044,1.029914,1.030983,1.033716,1.029146,1.036942,1.034100,1.036009,1.032870,1.014081,1.016428,1.014428,1.011377,1.009858,1.017281,1.016600,1.016465,1.038175,1.013018,1.024856,1.038283,1.014211,1.023343,1.012081,1.022813,1.014578,1.035889,1.000030,1.015148,1.015108,1.013529,1.006279,1.009487,1.025942,1.036959,1.038283,1.038175,1.036189,1.023343,1.019623,1.014578,1.014578,1.014578,1.015108
5,1.0,1.033112,1.034868,1.035643,1.038229,1.033948,1.041445,1.038669,1.040521,1.037410,1.018042,1.021488,1.018091,1.014513,1.012952,1.021778,1.021152,1.020679,1.042905,1.016913,1.029817,1.043014,1.018224,1.028436,1.015578,1.028192,1.018963,1.040645,1.000036,1.018931,1.019787,1.017055,1.008187,1.012087,1.030446,1.041465,1.043014,1.042905,1.040982,1.028436,1.024541,1.018963,1.018963,1.018963,1.019787
6,1.0,1.042804,1.044692,1.044627,1.046679,1.043225,1.049993,1.047330,1.049622,1.046506,1.026597,1.037886,1.026420,1.022462,1.020236,1.031302,1.032551,1.030722,1.049968,1.027775,1.039567,1.050000,1.029311,1.040060,1.024157,1.038782,1.030758,1.048786,1.000060,1.027257,1.034532,1.025314,1.013170,1.019115,1.039110,1.050000,1.050000,1.049968,1.049566,1.040060,1.036365,1.030758,1.030758,1.030758,1.034532
7,1.0,1.046479,1.048968,1.047569,1.046355,1.046408,1.049997,1.048409,1.049895,1.049979,1.030723,1.050000,1.030276,1.026080,1.023641,1.036210,1.041031,1.038594,1.050000,1.034776,1.045080,1.050000,1.036189,1.049987,1.028313,1.045430,1.038137,1.048632,1.000072,1.031135,1.045881,1.029120,1.013809,1.021219,1.042762,1.050000,1.050000,1.049998,1.050000,1.049955,1.046204,1.038136,1.038137,1.038137,1.045881
8,1.0,1.046608,1.049604,1.048185,1.046184,1.046729,1.049996,1.048246,1.049996,1.050000,1.030608,1.050000,1.030153,1.025725,1.023357,1.036566,1.040711,1.038098,1.050000,1.034318,1.045392,1.050000,1.035697,1.050000,1.027981,1.046349,1.037473,1.048207,1.000072,1.031113,1.045676,1.028942,1.012741,1.020369,1.042045,1.050000,1.050000,1.049998,1.050000,1.049983,1.046345,1.037473,1.037473,1.037473,1.045676
9,1.0,1.046482,1.049515,1.048168,1.046207,1.046628,1.049996,1.048332,1.049996,1.050000,1.030431,1.050000,1.029904,1.025521,1.023210,1.036305,1.040727,1.038006,1.050000,1.034325,1.045206,1.050000,1.035632,1.050000,1.027806,1.046080,1.037504,1.048254,1.00007

In [148]:
# GET LOAD P_MW

bus_loads = net.load.groupby("bus").apply(lambda x: x.index.tolist()) # Group loads by bus

load_p_mw = get_aggregated_x(0, bus_loads, get_from_controller)

load_p_mw = load_p_mw.iloc[start_time: start_time + total_time].reset_index(drop=True)

# # Print load P_MW for forecasting:
# load_p_mw.to_csv(out_path + "/total_load.csv")

swap_slack(load_p_mw, slack_bus)

load_p_mw.head(10)

C:\Users\bjorn\AppData\Local\Temp\ipykernel_30188\3292445622.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bus_loads = net.load.groupby("bus").apply(lambda x: x.index.tolist()) # Group loads by bus


bus,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
0,0,0,184.774800,194.042552,153.960297,64.562503,247.228259,0,95.033123,105.918116,374.073824,177.913665,285.542397,117.884773,157.179716,219.079873,317.529743,705.913250,0,252.705323,0,0,914.847691,125.847626,182.968558,148.455061,702.369036,0,196.474660,1189.367712,326.231379,142.771198,1127.555085,185.502541,256.281400,0,0,0,0,0,0,0,0,0,0
1,0,0,185.468888,189.550898,154.538634,64.805025,242.470070,0,95.390106,101.888276,365.392054,175.014164,286.615009,118.327596,157.770147,216.638302,318.722512,689.615014,0,253.654586,0,0,892.537180,126.320360,181.304344,145.173863,693.500929,0,197.212698,1167.965964,327.456835,143.307504,1104.954841,180.033735,246.782497,0,0,0,0,0,0,0,0,0,0
2,0,0,185.613922,186.505806,154.659480,64.855702,239.000902,0,95.464699,99.330671,359.190409,172.767503,286.839137,118.420126,157.893520,215.500335,318.971748,678.463338,0,253.852939,0,0,877.355627,126.419141,180.504386,142.778619,686.972861,0,197.366915,1149.588508,327.712901,143.419568,1085.965473,177.705312,240.674788,0,0,0,0,0,0,0,0,0,0
3,0,0,186.173336,187.190540,155.125602,65.051168,239.963344,0,95.752417,99.690152,360.687336,173.485601,287.703630,118.777027,158.369389,215.816740,319.933084,680.990911,0,254.618017,0,0,880.649365,126.800150,180.808473,143.389715,689.321750,0,197.961751,1155.644511,328.700582,143.851815,1091.824256,177.611804,241.580223,0,0,0,0,0,0,0,0,0,0
4,0,0,188.410994,193.538088,156.990091,65.833032,247.472194,0,96.903286,104.293423,373.138077,178.503875,291.161603,120.204635,160.272866,220.391255,323.778431,703.999845,0,257.678327,0,0,911.349012,128.324189,184.408302,148.231679,706.597696,0,200.341097,1192.287426,332.651307,145.580802,1128.364486,183.487208,252.567729,0,0,0,0,0,0,0,0,0,0
5,0,0,198.749805,225.856049,165.604720,69.445535,286.000262,0,102.220729,127.871756,437.475922,204.387327,307.138721,126.800709,169.067633,241.181138,341.545354,821.509516,0,271.818093,0,0,1068.398714,135.365814,200.791558,173.324096,793.125531,0,211.334556,1387.643704,350.905118,153.569360,1323.722298,209.980134,309.018037,0,0,0,0,0,0,0,0,0,0
6,0,0,206.581609,307.711045,172.130431,72.182060,384.808111,0,106.248772,191.593022,609.730135,271.220147,319.241627,131.797334,175.729801,265.505916,355.004066,1122.911767,0,282.529178,0,0,1475.772279,140.699950,219.379724,240.921722,986.489910,0,219.662267,1957.864201,364.732655,159.620814,1903.533726,246.246458,462.913820,0,0,0,0,0,0,0,0,0,0
7,0,0,209.140413,383.584516,174.262508,73.076136,478.086968,0,107.564812,251.145219,774.277772,335.073741,323.195884,133.429830,177.906461,275.496466,359.401291,1404.135603,0,286.028699,0,0,1857.012484,142.442717,226.924586,305.875696,1159.422568,0,222.383093,2532.792257,369.250381,161.597942,2489.648262,261.954259,607.640693,0,0,0,0,0,0,0,0,0,0
8,0,0,209.865580,406.108382,174.866741,73.329518,505.505620,0,107.937779,268.938542,822.645115,353.717425,324.316523,133.892480,178.523329,279.465036,360.647468,1487.451738,0,287.020466,0,0,1969.971286,142.936618,229.881990,324.914489,1210.650507,0,223.154178,2698.216890,370.530709,162.158262,2658.443370,268.553743,650.768219,0,0,0,0,0,0,0,0,0,0
9,0,0,209.679108,416.439758,174.711367,73.264363,518.484004,0,107.841873,277.069993,845.759593,362.725626,324.028359,133.773513,178.364706,279.071406,360.327022,1526.008914,0,286.765440,0,0,2022.369067,142.809615,229.573057,334.097589,1233.499622,0,222.955899,2783.439627,370.201482,162.014179,2745.440883,268.040667,670.667291,0,0,0,0,0,0,0,0,0,0


In [149]:
# GET LOAD Q_MVAR

load_q_mvar = load_p_mw.mul(p_to_q)

# load_q_mvar.head(50)

In [150]:
# GET SGEN P_MW

sgen_active = net.sgen.loc[net.sgen["in_service"] == True] # Drop unused sgens
bus_sgens = sgen_active.groupby("bus").apply(lambda x: x.index.tolist()) # Group sgens by bus

res_sgen_p_mw = pd.read_csv(out_path + "/res_sgen/p_mw.csv") # Read from simulation result

sgen_p_mw = get_aggregated_x(res_sgen_p_mw, bus_sgens, get_from_simulation) # Get bus aggregated sgen

swap_slack(sgen_p_mw, slack_bus)

sgen_p_mw.head(50)

C:\Users\bjorn\AppData\Local\Temp\ipykernel_30188\723073240.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bus_sgens = sgen_active.groupby("bus").apply(lambda x: x.index.tolist()) # Group sgens by bus


bus,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
0,0,0,0,18.962741,0,41.025113,31.331846,0,15.893070,2.718911,0.383358,10.738105,4.284659,2.509917,0,12.016477,0,2.274504,0,0,0,84.449375,0.979548,0,0,58.959378,0.356474,0,0,22.620266,1.035621,0,1.277389,0,4.532618,0,0,0,0,0,0,0,0,0,0
1,0,0,0,17.615876,0,32.941412,41.824214,0,19.980879,2.709443,0.129634,15.191834,2.900660,2.476805,0,8.097282,0,1.801014,0,0,0,38.549461,1.646019,0,0,44.631309,0.425653,0,0,14.658355,1.864704,0,2.129324,0,4.000259,0,0,0,0,0,0,0,0,0,0
2,0,0,0,11.558110,0,18.895813,49.417614,0,22.467116,2.348256,0.068982,20.050999,5.209552,5.178626,0,4.209387,0,1.216797,0,0,0,21.993624,3.677299,0,0,24.295891,0.487776,0,0,7.728606,2.917015,0,8.232676,0,2.879184,0,0,0,0,0,0,0,0,0,0
3,0,0,0,8.498079,0,10.266055,44.730267,0,19.972168,1.598577,0.326245,25.805955,12.549812,11.800203,0,3.346225,0,0.939706,0,0,0,24.411264,7.897133,0,0,12.677858,0.536576,0,0,9.208153,4.423040,0,20.578988,0,1.643538,0,0,0,0,0,0,0,0,0,0
4,0,0,0,11.045701,0,13.033721,36.143980,0,15.758490,0.982684,1.156841,34.320381,22.628434,20.728658,0,6.786727,0,1.268451,0,0,0,27.207727,13.699646,0,0,17.825050,0.584168,0,0,17.406658,6.417297,0,37.555506,0,1.077664,0,0,0,0,0,0,0,0,0,0
5,0,0,0,16.766447,0,22.646276,34.974544,0,13.827317,0.833529,2.218945,44.907753,32.968417,29.311481,0,12.804818,0,2.112199,0,0,0,37.121469,19.306713,0,0,33.377312,0.598368,0,0,26.777029,8.631971,0,52.987776,0,1.364369,0,0,0,0,0,0,0,0,0,0
6,0,0,0,22.876506,0,34.798007,41.005737,0,14.541684,1.263007,3.882239,55.599199,40.874903,35.646295,0,19.253865,0,4.412408,0,0,0,53.557790,28.702081,0,0,53.743552,3.802047,0,0,35.443622,10.916707,0,66.414607,0,2.588913,0,0,0,0,0,0,0,0,0,0
7,0,0,0,41.748995,0,41.060965,59.648206,0,15.760052,10.777707,21.187390,72.688264,44.759865,37.833408,0,22.894430,0,28.929631,0,0,0,76.287865,98.634884,0,0,76.082393,46.143464,0,0,73.441841,10.085764,0,107.026613,0,17.156731,0,0,0,0,0,0,0,0,0,0
8,0,0,0,70.215968,0,31.504007,80.371115,0,13.582621,26.614701,53.648786,93.827584,39.017257,31.909744,0,17.924017,0,62.641051,0,0,0,102.643748,198.770990,0,0,82.414514,114.885334,0,0,147.192030,5.718794,0,167.906735,0,40.631670,0,0,0,0,0,0,0,0,0,0
9,0,0,0,99.254431,0,22.062030,101.249725,0,11.190302,49.856283,92.432327,113.025779,35.545514,27.882086,0,12.934899,0,111.365816,0,0,0,121.116547,291.566530,0,0,85.516257,162.669519,0,0,197.094845,3.583982,0,231.870407,0,68.998626,0,0,0,0,0,0,0,0,0,0


In [151]:
# GET SGEN Q_MVAR

res_sgen_q_mvar = pd.read_csv(out_path + "/res_sgen/q_mvar.csv") # Read from simulation result

sgen_q_mvar = get_aggregated_x(res_sgen_q_mvar, bus_sgens, get_from_simulation) # Get bus aggregated sgen

swap_slack(sgen_q_mvar, slack_bus)

sgen_q_mvar.head(50)

bus,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
0,0,0,0,1.184927e-16,0,4.647802e-17,1.595940e-16,0,7.042449e-17,2.047111e-16,1.072370e-16,7.663641e-17,2.011500e-16,3.173749e-16,0,-6.746415e-17,0,-1.182997e-14,0,0,0,-2.699322e-18,1.204868e-16,0,0,2.412419e-16,6.224963e-16,0,0,6.104100e-16,8.983986e-17,0,-4.733288e-14,0,3.836540e-15,0,0,0,0,0,0,0,0,0,0
1,0,0,0,-4.944142e-16,0,-3.331296e-16,-6.740895e-16,0,-3.040954e-16,-8.468436e-16,-4.525880e-16,-3.256673e-16,-8.628743e-16,-1.303043e-15,0,2.860991e-16,0,4.680507e-14,0,0,0,1.202520e-17,-5.069814e-16,0,0,-1.007050e-15,-2.618991e-15,0,0,-2.587222e-15,-3.832537e-16,0,1.420377e-13,0,-1.730224e-14,0,0,0,0,0,0,0,0,0,0
2,0,0,0,-8.500886e-18,0,-1.622396e-18,-1.230589e-17,0,-5.603001e-18,-1.484965e-17,-8.177891e-18,-5.613179e-18,-1.527852e-17,-1.746879e-17,0,5.708325e-18,0,1.365844e-15,0,0,0,2.007771e-19,-9.188503e-18,0,0,-1.950906e-17,-4.433681e-17,0,0,-4.713937e-17,-6.592670e-18,0,6.127475e-15,0,-3.629893e-16,0,0,0,0,0,0,0,0,0,0
3,0,0,0,8.734914e-17,0,-1.753490e-17,1.309201e-16,0,5.901201e-17,1.563274e-16,8.420251e-17,5.815462e-17,1.553515e-16,1.253635e-16,0,-6.306754e-17,0,-1.433299e-14,0,0,0,-1.844014e-18,9.508542e-17,0,0,2.143315e-16,4.519298e-16,0,0,4.889480e-16,6.767743e-17,0,-2.968883e-14,0,3.750227e-15,0,0,0,0,0,0,0,0,0,0
4,0,0,0,-4.004506e-16,0,1.036840e-16,-6.022674e-16,0,-2.658220e-16,-7.308623e-16,-3.855982e-16,-2.658972e-16,-6.996209e-16,-5.392163e-16,0,2.889143e-16,0,5.835294e-14,0,0,0,7.794386e-18,-4.397898e-16,0,0,-9.988037e-16,-2.075896e-15,0,0,-2.224959e-15,-3.064825e-16,0,6.089808e-14,0,-1.521657e-14,0,0,0,0,0,0,0,0,0,0
5,0,0,0,-1.410642e-14,0,3.906439e-15,-2.071208e-14,0,-8.320162e-15,-2.711763e-14,-1.308233e-14,-8.990550e-15,-2.236776e-14,-1.952512e-14,0,9.557546e-15,0,2.177738e-12,0,0,0,1.667396e-16,-1.538128e-14,0,0,-3.523633e-14,-7.216302e-14,0,0,-7.397428e-14,-1.001846e-14,0,3.411754e-12,0,-3.071332e-13,0,0,0,0,0,0,0,0,0,0
6,0,0,0,2.911531e-14,0,-5.649820e-14,4.056233e-14,0,1.337399e-14,6.306022e-14,2.468540e-14,1.532355e-14,3.490046e-14,3.324352e-14,0,-2.104938e-14,0,-1.399488e-13,0,0,0,1.497394e-16,2.928404e-14,0,0,7.698108e-14,1.392523e-13,0,0,1.451246e-13,1.503389e-14,0,5.780984e-12,0,-2.109457e-13,0,0,0,0,0,0,0,0,0,0
7,0,0,0,-2.930696e-17,0,-5.439417e-16,-3.743416e-17,0,-1.826287e-17,-8.897926e-17,-1.008142e-17,8.077632e-18,1.652464e-17,1.022406e-16,0,5.085116e-17,0,-1.456008e-14,0,0,0,-2.470332e-18,-3.773235e-18,0,0,-1.152743e-16,-6.539315e-18,0,0,-9.437694e-17,3.005548e-17,0,-2.501468e-14,0,-3.478806e-15,0,0,0,0,0,0,0,0,0,0
8,0,0,0,1.112027e-17,0,5.192940e-16,9.547641e-18,0,4.533744e-18,2.311624e-17,1.493319e-18,5.779697e-19,-1.063816e-17,-7.646335e-17,0,-6.742099e-18,0,-2.724307e-15,0,0,0,1.248582e-18,2.610355e-18,0,0,9.213406e-18,4.832824e-17,0,0,2.348991e-17,-1.478607e-18,0,-8.791405e-14,0,4.139013e-15,0,0,0,0,0,0,0,0,0,0
9,0,0,0,4.708601e-16,0,4.621087e-15,5.500768e-16,0,1.978329e-16,1.035236e-15,-3.491933e-16,-2.330863e-17,-1.053209e-15,-4.530792e-15,0,-3.333428e-16,0,2.590512e-13,0,0,0,9.839733e-17,-1.560278e-16,0,0,2.055129e-16,1.946608e-15,0,0,4.562455e-16,-7.312801e-17,0,1.112106e-14,0,8.403553e-14,0,0,0,0,0,0,0,0,0,0


In [152]:
# GET BUS ANGLE

res_va_degree = pd.read_csv(out_path + "/res_bus/va_degree.csv")
va_degree = res_va_degree.drop(columns=["Unnamed: 0"])

swap_slack(va_degree, slack_bus)

va_degree.head(20)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
0,0.0,-2.111440,-1.972398,-1.669874,0.826488,-1.900822,1.457308,-0.207749,1.106993,-0.840886,-2.060252,-2.010414,-2.117140,-2.330412,-1.828996,-2.126613,-8.174057,-7.930780,2.816640,-5.423368,-2.395907,2.917390,-3.163882,-5.480702,-2.212364,-2.188848,-7.153927,1.291528,-0.000104,-2.164293,-2.797227,-2.221272,-2.461465,-2.442939,-2.810730,1.466238,2.917390,2.816640,1.289812,-5.480702,-7.940843,-7.153926,-7.153926,-7.153927,-2.797227
1,0.0,-2.151758,-2.005481,-1.738048,0.729501,-1.979272,1.363932,-0.277579,1.018782,-0.889814,-2.081156,-2.025511,-2.125367,-2.325722,-1.856150,-2.146504,-8.108861,-7.863397,2.681748,-5.375286,-2.416419,2.780000,-3.132303,-5.454669,-2.219022,-2.207818,-7.085586,1.187851,-0.000101,-2.170219,-2.795174,-2.227010,-2.439114,-2.424791,-2.826348,1.372555,2.780000,2.681748,1.186287,-5.454669,-7.873161,-7.085584,-7.085585,-7.085586,-2.795174
2,0.0,-2.204429,-2.045304,-1.839483,0.693181,-2.097326,1.326325,-0.333827,0.986219,-0.935558,-2.103805,-2.053415,-2.126633,-2.320069,-1.885840,-2.168313,-8.057624,-7.811106,2.629077,-5.341681,-2.437038,2.726396,-3.114120,-5.431860,-2.228672,-2.228773,-7.035008,1.147325,-0.000100,-2.172398,-2.804375,-2.226857,-2.416109,-2.406597,-2.834273,1.334805,2.726396,2.629077,1.145836,-5.431860,-7.820659,-7.035008,-7.035008,-7.035008,-2.804375
3,0.0,-2.253385,-2.077517,-1.950677,0.607865,-2.206432,1.236136,-0.424159,0.893102,-1.013309,-2.110346,-2.059019,-2.124558,-2.316561,-1.893587,-2.174560,-8.105865,-7.862570,2.551444,-5.365577,-2.461069,2.648928,-3.124797,-5.478890,-2.233213,-2.235382,-7.070968,1.075911,-0.000100,-2.176295,-2.811246,-2.224642,-2.408885,-2.400541,-2.875425,1.244718,2.648928,2.551444,1.074450,-5.478890,-7.872170,-7.070968,-7.070968,-7.070968,-2.811246
4,0.0,-2.344592,-2.191305,-2.117378,0.390589,-2.338266,1.012120,-0.620269,0.662257,-1.201714,-2.086799,-2.032239,-2.127159,-2.320824,-1.860670,-2.152558,-8.336159,-8.101218,2.348968,-5.494638,-2.503249,2.446576,-3.187343,-5.661258,-2.229549,-2.215489,-7.262996,0.888298,-0.000103,-2.184963,-2.821130,-2.228238,-2.428404,-2.416778,-2.993768,1.020907,2.446576,2.348968,0.886836,-5.661258,-8.111251,-7.262995,-7.262996,-7.262996,-2.821130
5,0.0,-2.714788,-2.709327,-2.807585,-0.366570,-2.886042,0.226859,-1.369066,-0.122891,-1.952825,-1.914752,-1.842809,-2.167690,-2.372249,-1.597080,-1.969405,-9.377323,-9.172811,1.618420,-6.093911,-2.614864,1.715741,-3.481292,-6.440197,-2.172885,-2.041135,-8.152751,0.211640,-0.000126,-2.241697,-2.835862,-2.273819,-2.602926,-2.559584,-3.443562,0.236129,1.715741,1.618420,0.210009,-6.440197,-9.185032,-8.152751,-8.152751,-8.152751,-2.835862
6,0.0,-5.280879,-5.303907,-5.495729,-2.151537,-5.500715,-1.638541,-3.719886,-1.629214,-4.540821,-3.940570,-1.955416,-4.426587,-4.139847,-2.818893,-3.985296,-13.316948,-13.289853,-0.411984,-8.924575,-5.121023,-0.314731,-5.661085,-9.954976,-3.854818,-4.306938,-11.654705,-1.950580,-0.000208,-4.669825,-3.751475,-4.427951,-4.247749,-4.254691,-6.295728,-1.631164,-0.314731,-0.411983,-1.954329,-9.954976,-13.307535,-11.654705,-11.654705,-11.654705,-3.751475
7,0.0,-2.410867,-1.821562,-1.119889,4.191617,-1.704116,5.602485,1.893946,5.191916,0.353140,-4.533977,1.127933,-5.264919,-4.832413,-3.219294,-4.227807,-13.242858,-13.050212,6.273579,-9.084768,-3.442992,6.372293,-5.703938,-8.809962,-4.335159,-3.234313,-11.937789,3.723426,-0.000250,-5.558824,-1.875926,-5.235093,-5.119762,-5.075566,-4.149099,5.603411,6.372293,6.273585,3.716857,-8.809792,-13.073876,-11.937789,-11.937789,-11.937789,-1.875926
8,0.0,-0.925638,-0.147056,0.865197,6.779913,0.060355,8.289098,4.185024,7.827458,2.470792,-4.368697,1.288278,-5.150459,-4.799550,-3.146995,-3.857697,-12.581801,-12.288397,9.179084,-8.782161,-2.323738,9.279366,-5.539320,-7.771665,-4.247253,-2.347287,-11.487581,6.246583,-0.000249,-5.408241,-1.720061,-5.138469,-5.176642,-5.105699,-2.813505,8.291313,9.279366,9.17908

In [153]:
# GET SLACK WEIGHT (AND SLACK)

slack_active = net.ext_grid.loc[net.ext_grid["in_service"] == True]

slack_weight = [1 if i in slack_active.bus.to_list() else 0 for i in total_bus_numbers]
    
slack_weight[0], slack_weight[slack_bus] = slack_weight[slack_bus], slack_weight[0]

slack = slack_weight # Slack yes or no

print(slack_weight)

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]


In [154]:
# GET EMPTY BUSES

active_bus = set(net.load.bus.to_list())
active_bus.update(gen_active.bus.to_list())
active_bus.update(sgen_active.bus.to_list())
active_bus.update(slack_active.bus.to_list())

empty_bus = [bus for bus in total_bus_numbers if bus not in active_bus]

for i in range(len(empty_bus)):
    if empty_bus[i] == 0:
        empty_bus[i] = slack_bus
    elif empty_bus[i] == slack_bus:
        empty_bus[i] = 0

print(empty_bus)

[35, 1, 7, 20, 27, 36]


In [155]:
# GET BUS TYPE

# 2 if PV (or slack), 3 if PQ (or empty/not in service --> P=0, Q=0)
bus_type = pd.Series([2 if (i in bus_gens.index or i in slack_active.bus.to_list()) 
                   else 3 for i in total_bus_numbers])

bus_type[0], bus_type[slack_bus] = 1, bus_type[0] # Main slack is 1

print(bus_type)

0     1
1     3
2     2
3     2
4     3
5     3
6     2
7     3
8     2
9     2
10    2
11    2
12    2
13    2
14    2
15    2
16    2
17    3
18    2
19    3
20    3
21    2
22    2
23    2
24    2
25    2
26    2
27    3
28    2
29    2
30    2
31    3
32    3
33    2
34    3
35    3
36    3
37    2
38    2
39    2
40    2
41    2
42    2
43    2
44    2
dtype: int64


In [156]:
# JOIN LOAD AND SGEN

load_sgen_p_mw = load_p_mw.subtract(sgen_p_mw)
load_sgen_q_mvar = load_q_mvar.subtract(sgen_q_mvar)

load_sgen_p_mw.head(5)

bus,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
0,0,0,184.774800,175.079811,153.960297,23.537389,215.896413,0,79.140054,103.199205,373.690466,167.175560,281.257737,115.374856,157.179716,207.063396,317.529743,703.638746,0,252.705323,0,-84.449375,913.868143,125.847626,182.968558,89.495683,702.012562,0,196.474660,1166.747446,325.195758,142.771198,1126.277696,185.502541,251.748782,0,0,0,0,0,0,0,0,0,0
1,0,0,185.468888,171.935022,154.538634,31.863613,200.645856,0,75.409227,99.178833,365.262420,159.822330,283.714349,115.850791,157.770147,208.541020,318.722512,687.814000,0,253.654586,0,-38.549461,890.891161,126.320360,181.304344,100.542554,693.075275,0,197.212698,1153.307609,325.592131,143.307504,1102.825517,180.033735,242.782238,0,0,0,0,0,0,0,0,0,0
2,0,0,185.613922,174.947696,154.659480,45.959889,189.583288,0,72.997584,96.982415,359.121427,152.716505,281.629585,113.241500,157.893520,211.290948,318.971748,677.246541,0,253.852939,0,-21.993624,873.678328,126.419141,180.504386,118.482728,686.485085,0,197.366915,1141.859902,324.795886,143.419568,1077.732797,177.705312,237.795604,0,0,0,0,0,0,0,0,0,0
3,0,0,186.173336,178.692460,155.125602,54.785113,195.233077,0,75.780248,98.091574,360.361091,147.679646,275.153818,106.976824,158.369389,212.470514,319.933084,680.051206,0,254.618017,0,-24.411264,872.752232,126.800150,180.808473,130.711857,688.785173,0,197.961751,1146.436357,324.277542,143.851815,1071.245268,177.611804,239.936684,0,0,0,0,0,0,0,0,0,0
4,0,0,188.410994,182.492386,156.990091,52.799311,211.328214,0,81.144796,103.310739,371.981235,144.183494,268.533169,99.475976,160.272866,213.604528,323.778431,702.731395,0,257.678327,0,-27.207727,897.649366,128.324189,184.408302,130.406629,706.013528,0,200.341097,1174.880768,326.234009,145.580802,1090.808979,183.487208,251.490065,0,0,0,0,0,0,0,0,0,0


In [159]:
# GET MAX AND MIN GEN MVAR

min_gen_q_mvar = gen_active.groupby("bus")["min_q_mvar"].sum() # Bus-wise sum of gen min q_mvar
slack_min_gen_qmvar = slack_active["min_q_mvar"].set_axis(slack_active["bus"]) # Slack bus min q_mvar
min_gen_q_mvar = pd.concat([min_gen_q_mvar, slack_min_gen_qmvar])

min_gen_q_mvar = min_gen_q_mvar.reindex(range(total_bus_numbers[0], total_bus_numbers[-1] + 1), fill_value=None) # Fill missing buses with none
min_gen_q_mvar[0], min_gen_q_mvar[slack_bus] = min_gen_q_mvar[slack_bus], min_gen_q_mvar[0] # Swap slack bus

max_gen_q_mvar = gen_active.groupby("bus")["max_q_mvar"].sum() # Bus-wise sum of gen max q_mvar
slack_max_gen_qmvar = slack_active["max_q_mvar"].set_axis(slack_active["bus"]) # Slack bus min q_mvar
max_gen_q_mvar = pd.concat([max_gen_q_mvar, slack_max_gen_qmvar])

max_gen_q_mvar = max_gen_q_mvar.reindex(range(total_bus_numbers[0], total_bus_numbers[-1] + 1), fill_value=None) # Fill missing buses with none
max_gen_q_mvar[0], max_gen_q_mvar[slack_bus] = max_gen_q_mvar[slack_bus], max_gen_q_mvar[0] # Swap slack bus

min_gen_q_mvar

bus
0    -5000.0
1        NaN
2    -1000.0
3    -2000.0
4        NaN
5        NaN
6    -4000.0
7        NaN
8    -2000.0
9    -1000.0
10   -3000.0
11   -4000.0
12   -4000.0
13   -1000.0
14   -1000.0
15   -4000.0
16   -1000.0
17       NaN
18   -2000.0
19       NaN
20       NaN
21   -4000.0
22   -5000.0
23   -2000.0
24   -3000.0
25   -2000.0
26   -1000.0
27       NaN
28   -7000.0
29   -3000.0
30   -2000.0
31       NaN
32       NaN
33   -1000.0
34       NaN
35       NaN
36       NaN
37   -5000.0
38   -5000.0
39   -5000.0
40   -5000.0
41   -5000.0
42   -5000.0
43   -5000.0
44   -5000.0
Name: min_q_mvar, dtype: float64

In [160]:
# GET USER SOLAR AND WIND BUSSES

extra_bus = pd.Series([2, 1.0, None, None, None, 0, 0, 1000.0, -1000.0, 0, 0])
extra_buses = pd.DataFrame([extra_bus, extra_bus])
extra_buses.columns = ["Type", "PU Volt", "Angle (Deg)", "Gen MW", "Gen Mvar", "Load MW", "Load Mvar", 
                       "Gen Mvar(max)", "Gen Mvar(min)", "Slack yes,no", "Slack Weight"]
extra_buses

,Type,PU Volt,Angle (Deg),Gen MW,Gen Mvar,Load MW,Load Mvar,Gen Mvar(max),Gen Mvar(min),"Slack yes,no",Slack Weight
0,2.0,1.0,NaN,NaN,NaN,0.0,0.0,1000.0,-1000.0,0.0,0.0
1,2.0,1.0,NaN,NaN,NaN,0.0,0.0,1000.0,-1000.0,0.0,0.0


In [161]:
# TRANSFORM TO P.U.

def to_pu(power, sbase=100.0):
    return np.divide(power, sbase)

In [162]:
# GETTING ALL DATA FOR ONE TIMESTEP

nones = [None for _ in total_bus_numbers]

def get_timestep(t):
    timestep = pd.DataFrame(data={"Type": bus_type, "PU Volt": vm_pu.iloc[t].values, "Angle (Deg)": va_degree.iloc[t].values, 
    "Gen MW": to_pu(gen_p_mw.iloc[t].values), "Gen Mvar": nones, "Load MW": to_pu(load_sgen_p_mw.iloc[t].values), 
    "Load Mvar": to_pu(load_sgen_q_mvar.iloc[t].values), "Gen Mvar(max)": to_pu(max_gen_q_mvar), 
    "Gen Mvar(min)": to_pu(min_gen_q_mvar), "Slack yes,no": slack, "Slack Weight": slack_weight})
    
    timestep = pd.concat([timestep, extra_buses], axis=0, ignore_index=True) # Add solar and wind bus
    
    return timestep

get_timestep(0).head(50)


,Type,PU Volt,Angle (Deg),Gen MW,Gen Mvar,Load MW,Load Mvar,Gen Mvar(max),Gen Mvar(min),"Slack yes,no",Slack Weight
0,1.0,1.000000,0.000000,0.000000,None,0.000000,0.000000e+00,50.0,-50.0,1.0,1.0
1,3.0,1.029049,-2.111440,0.000000,None,0.000000,0.000000e+00,NaN,NaN,0.0,0.0
2,2.0,1.031064,-1.972398,1.073640,None,1.847748,6.073254e-01,10.0,-10.0,0.0,0.0
3,2.0,1.032532,-1.669874,0.000001,None,1.750798,6.377870e-01,20.0,-20.0,0.0,0.0
4,3.0,1.035399,0.826488,0.000000,None,1.539603,5.060430e-01,NaN,NaN,0.0,0.0
5,3.0,1.030620,-1.900822,0.000000,None,0.235374,2.122067e-01,NaN,NaN,0.0,0.0
6,2.0,1.038637,1.457308,0.000074,None,2.158964,8.126000e-01,40.0,-40.0,0.0,0.0
7,3.0,1.035664,-0.207749,0.000000,None,0.000000,0.000000e+00,NaN,NaN,0.0,0.0
8,2.0,1.037715,1.106993,0.500028,None,0.791401,3.123588e-01,20.0,-20.0,0.0,0.0
9,2.0,1.034367,-0.840886,0.480000,None,1.031992,3.481360e-01,10.0,-10.0,0.0,0.0


In [163]:
# CREATING THE CSV WITH ALL TIMESTEPS

def create_csv(t_init=0, t_duration=total_time, index=True):
    result = pd.concat([get_timestep(t) for t in range(t_init, t_init + t_duration)], axis=0)
    
    result.to_csv(out_path + "/busInputData.csv", index=index)

In [80]:
# GET NON-CONVERGED TIMESTEPS INDEXES

non_conv_mask = (vm_pu == 0).all(axis=1)
non_conv_idx = vm_pu.index[non_conv_mask].to_list() # Non converged timesteps index

conv_periods = [non_conv_idx[i+1] - non_conv_idx[i] for i in range(len(non_conv_idx) - 1)] # Periods of convergence
from_to_conv = [[non_conv_idx[i], non_conv_idx[i+1]] for i in range(len(non_conv_idx) - 1)]
convergences = dict(zip(conv_periods, from_to_conv))

conv_periods_sorted = conv_periods
conv_periods_sorted.sort(reverse=True)
conv = {i: convergences[i] for i in conv_periods_sorted} # {Length period of convergence: [start, end]}

print(conv)

# Result is {186: [7982, 8168], 149: [4829, 4978], 141: [8364, 8505], 131: [6770, 6901], 121: [5959, 6080], 113: [7391, 7504], 112: [7746, 7858], 110: [5775, 5885], 106: [728, 834], 105: [6080, 6185], 104: [6911, 7015], 103: [136, 239], 100: [6406, 6506], 98: [3519, 3617], 96: [1329, 1425], 93: [4173, 4266], 91: [8271, 8362], 89: [1885, 1974], 84: [1069, 1153], 82: [7266, 7348], 80: [3711, 3791], 79: [3440, 3519], 77: [6563, 6640], 76: [8511, 8587], 75: [7015, 7090], 74: [7136, 7210], 72: [5141, 5213], 70: [426, 496], 68: [7911, 7979], 63: [5078, 5141], 62: [7576, 7638], 61: [2205, 2266], 60: [8587, 8647], 59: [3835, 3894], 56: [7210, 7266], 55: [1561, 1616], 54: [9, 63], 53: [7638, 7691], 52: [6326, 6378], 51: [8220, 8271], 50: [4743, 4793], 48: [7691, 7739], 47: [2933, 2980], 46: [919, 965], 45: [3234, 3279], 44: [7504, 7548], 43: [7348, 7391], 42: [2987, 3029], 41: [5653, 5694], 40: [4357, 4397], 38: [4077, 4115], 37: [2474, 2511], 36: [4041, 4077], 34: [7877, 7911], 33: [6506, 6539], 32: [6185, 6217], 31: [3898, 3929], 29: [5460, 5489], 28: [7548, 7576], 27: [6679, 6706], 26: [5312, 5338], 25: [5694, 5719], 24: [6640, 6664], 23: [7113, 7136], 22: [5593, 5615], 20: [4637, 4657], 19: [613, 632], 18: [8174, 8192], 17: [5885, 5902], 16: [8647, 8663], 15: [8205, 8220], 14: [5271, 5285], 13: [8192, 8205], 12: [5489, 5501], 11: [6759, 6770], 10: [6901, 6911], 9: [5758, 5767], 8: [6398, 6406], 7: [6296, 6303], 6: [8168, 8174], 5: [8506, 8511], 4: [5348, 5352], 3: [5432, 5435], 2: [8362, 8364], 1: [8505, 8506]}

{186: [7982, 8168], 149: [4829, 4978], 141: [8364, 8505], 131: [6770, 6901], 121: [5959, 6080], 113: [7391, 7504], 112: [7746, 7858], 110: [5775, 5885], 106: [728, 834], 105: [6080, 6185], 104: [6911, 7015], 103: [136, 239], 100: [6406, 6506], 98: [3519, 3617], 96: [1329, 1425], 93: [4173, 4266], 91: [8271, 8362], 89: [1885, 1974], 84: [1069, 1153], 82: [7266, 7348], 80: [3711, 3791], 79: [3440, 3519], 77: [6563, 6640], 76: [8511, 8587], 75: [7015, 7090], 74: [7136, 7210], 72: [5141, 5213], 70: [426, 496], 68: [7911, 7979], 63: [5078, 5141], 62: [7576, 7638], 61: [2205, 2266], 60: [8587, 8647], 59: [3835, 3894], 56: [7210, 7266], 55: [1561, 1616], 54: [9, 63], 53: [7638, 7691], 52: [6326, 6378], 51: [8220, 8271], 50: [4743, 4793], 48: [7691, 7739], 47: [2933, 2980], 46: [919, 965], 45: [3234, 3279], 44: [7504, 7548], 43: [7348, 7391], 42: [2987, 3029], 41: [5653, 5694], 40: [4357, 4397], 38: [4077, 4115], 37: [2474, 2511], 36: [4041, 4077], 34: [7877, 7911], 33: [6506, 6539], 32: [6185

In [164]:
create_csv(index=False) # 6912/24 = 288th day 1st hour

In [104]:
# TESTING OPF

res_load_p_mw = pd.read_csv(out_path + "/res_load/p_mw.csv") # Read from simulation result
res_load_q_mvar = res_load_p_mw.mul(p_to_q)

res_gen_vm_pu = pd.read_csv(out_path + "/res_gen/vm_pu.csv")

res_ext_grid_vpu = res_vm_pu.iloc[:, 36:] # Wrong
res_ext_grid_vm_pu = pd.concat([res_ext_grid_vpu.iloc[:, 3:], res_ext_grid_vpu.iloc[:, :3]], axis=1)

res_ext_grid_vadeg = res_va_degree.iloc[:, 36:]
res_ext_grid_va_degree = pd.concat([res_ext_grid_vadeg.iloc[:, 3:], res_ext_grid_vadeg.iloc[:, :3]], axis=1)
# res_ext_grid_vm_pu.head()


def run_simulation(t_init=0, t_final=8760, function=pp.runpp):

    # init_load = net.load[["p_mw", "q_mvar"]]

    net.ext_grid["vm_pu"] = np.ones(len(net.ext_grid))
    net.ext_grid["va_degree"] = np.zeros(len(net.ext_grid))

    res_lp = []
    res_lq = []

    res_gp = []
    res_gq = []
    res_gvm = []
    res_gva = []

    res_sgp = []
    res_sgq = []

    res_bp = []
    res_bq = []
    res_bvm = []
    res_bva = []

    res_l = []

    for t in range(t_init, t_final):
        net.load["p_mw"] = res_load_p_mw.iloc[t, 1:].values
        net.load["q_mvar"] = res_load_q_mvar.iloc[t, 1:].values

        net.gen["p_mw"] = res_gen_p_mw.iloc[t, 1:].values
        net.gen["vm_pu"] = res_gen_vm_pu.iloc[t, 1:].values

        net.sgen["p_mw"] = res_sgen_p_mw.iloc[t, 1:].values
        net.sgen["q_mvar"] = res_sgen_q_mvar.iloc[t, 1:].values

        # print(res_ext_grid_vm_pu.iloc[t].values)

        net.ext_grid["vm_pu"] = res_ext_grid_vm_pu.iloc[t].values
        net.ext_grid["va_degree"] = res_ext_grid_va_degree.iloc[t].values

        function(net)

        res_lp.append(net.res_load["p_mw"].copy().to_frame().T) # load_p_mw
        res_lq.append(net.res_load["q_mvar"].copy().to_frame().T) # load_q_mvar

        res_gp.append(net.res_gen["p_mw"].copy().to_frame().T) # gen_p_mw
        res_gq.append(net.res_gen["q_mvar"].copy().to_frame().T) # gen_q_mvar
        res_gvm.append(net.res_gen["vm_pu"].copy().to_frame().T) # gen_vm_pu
        res_gva.append(net.res_gen["va_degree"].copy().to_frame().T) # gen_q_mvar

        res_sgp.append(net.res_sgen["p_mw"].copy().to_frame().T) # gen_p_mw
        res_sgq.append(net.res_sgen["q_mvar"].copy().to_frame().T) # gen_p_mw

        res_bp.append(net.res_bus["p_mw"].copy().to_frame().T) # gen_q_mvar
        res_bq.append(net.res_bus["q_mvar"].copy().to_frame().T) # gen_q_mvar
        res_bvm.append(net.res_bus["vm_pu"].copy().to_frame().T) # gen_q_mvar
        res_bva.append(net.res_bus["va_degree"].copy().to_frame().T) # gen_q_mvar

        res_l.append(net.res_line["loading_percent"].copy().to_frame().T) # gen_q_mvar

    
    res_lp = pd.concat(res_lp, axis=0)
    res_lq = pd.concat(res_lq, axis=0)

    res_gp = pd.concat(res_gp, axis=0)
    res_gq = pd.concat(res_gq, axis=0)
    res_gvm = pd.concat(res_gvm, axis=0)
    res_gva = pd.concat(res_gva, axis=0)

    res_sgp = pd.concat(res_sgp, axis=0)
    res_sgq = pd.concat(res_sgq, axis=0)

    res_bp = pd.concat(res_bp, axis=0)
    res_bq = pd.concat(res_bq, axis=0)
    res_bvm = pd.concat(res_bvm, axis=0)
    res_bva = pd.concat(res_bva, axis=0)

    res_l = pd.concat(res_l, axis=0)

    return {"load": [res_lp, res_lq], "gen": [res_gp, res_gq, res_gvm, res_gva], "sgen": [res_sgp, res_sgq], 
            "bus": [res_bp, res_bq, res_bvm, res_bva], "line": [res_l]}


results = run_simulation(t_init=6912, t_final=6912+48, function=pp.runpp)


In [105]:
results["line"][0].head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91
loading_percent,18.233950,18.233950,1.580832,1.580832,11.659965,11.659965,2.898469,2.898469,1.987191,1.987191,5.520081,5.520081,14.571593,14.527192,14.527192,8.340281,8.340281,4.165912,4.165912,5.612250,5.612250,1.152192,11.026057,11.026057,1.304492,1.304492,9.709734,9.709734,1.528564,1.528564,10.011686,10.011686,1.045385,1.045385,3.360951,3.360951,23.252188,23.252188,2.437585,2.437585,17.162399,17.162399,6.954519,6.954519,30.142651,30.142651,0.237214,0.237214,17.902232,17.902232,17.902232,3.487137,3.487137,4.257628,2.546404,2.546404,13.602280,13.602280,1.161652,3.250892,3.250892,5.934682,5.934682,17.311775,17.311775,0.145714,0.145714,0.392820,0.392820,10.339584,10.339584,1.253589,0.095489,0.095489,0.116904,0.116904,18.772271,18.772271,3.017186,3.017186,6.466725,6.466725,24.683051,24.683051,15.663267,15.663267,0.001809,0.001809,0.000905,0.000905,0.014955,0.014955
loading_percent,17.559955,17.559955,1.678299,1.678299,11.614363,11.614363,3.009403,3.009403,1.736706,1.736706,5.439425,5.439425,14.317521,14.275402,14.275402,8.038684,8.038684,4.257582,4.257582,5.512645,5.512645,1.136149,10.752762,10.752762,1.175143,1.175143,9.423780,9.423780,1.390250,1.390250,9.488018,9.488018,1.043797,1.043797,3.373837,3.373837,22.826321,22.826321,2.365634,2.365634,16.765755,16.765755,6.866590,6.866590,29.298525,29.298525,0.237187,0.237187,17.746960,17.746960,17.746960,3.219362,3.219362,3.881888,2.322789,2.322789,13.055100,13.055100,1.164285,3.271462,3.271462,5.931987,5.931987,17.081433,17.081433,0.145716,0.145716,0.437273,0.437273,10.497075,10.497075,0.853399,0.095549,0.095549,0.116903,0.116903,18.599088,18.599088,2.669798,2.669798,6.292027,6.292027,24.252527,24.252527,15.294711,15.294711,0.001809,0.001809,0.002025,0.002025,0.129480,0.129480
loading_percent,17.256068,17.256068,1.791888,1.791888,11.961747,11.961747,3.203115,3.203115,1.368100,1.368100,5.357513,5.357513,14.470198,14.429717,14.429717,7.904709,7.904709,4.303123,4.303123,5.487449,5.487449,1.119153,10.505172,10.505172,1.262884,1.262884,9.360288,9.360288,1.359558,1.359558,8.940545,8.940545,1.033519,1.033519,3.372632,3.372632,22.532125,22.532125,2.316245,2.316245,16.616712,16.616712,6.643155,6.643155,28.998483,28.998483,0.237153,0.237153,17.617855,17.617855,17.617855,3.011751,3.011751,3.503595,1.991492,1.991492,12.640271,12.640271,1.185332,3.273548,3.273548,5.924203,5.924203,16.904870,16.904870,0.145705,0.145705,0.487919,0.487919,10.676087,10.676087,0.275742,0.095488,0.095488,0.116897,0.116897,18.415123,18.415123,2.361599,2.361599,6.141454,6.141454,23.952292,23.952292,15.160935,15.160935,0.001809,0.001809,0.000906,0.000906,0.018238,0.018238
loading_percent,17.453758,17.453758,1.987572,1.987572,12.123150,12.123150,3.175176,3.175176,1.121131,1.121131,5.405465,5.405465,14.460286,14.418941,14.418941,7.728679,7.728679,4.207112,4.207112,5.259115,5.259115,1.113543,10.519124,10.519124,1.424544,1.424544,9.357539,9.357539,1.350440,1.350440,8.855882,8.855882,1.038423,1.038423,3.342693,3.342693,22.575699,22.575699,2.326611,2.326611,16.547680,16.547680,6.938577,6.938577,29.041899,29.041899,0.237156,0.237156,17.721952,17.721952,17.721952,3.038302,3.038302,3.449926,1.827380,1.827380,12.729735,12.729735,1.191475,3.547947,3.547947,5.989194,5.989194,17.024239,17.024239,0.145707,0.145707,0.499590,0.499590,10.721194,10.721194,0.248303,0.095489,0.095489,0.116898,0.116898,18.359242,18.359242,2.300812,2.300812,6.159402,6.159402,24.008148,24.008148,15.049607,15.049607,0.001809,0.001809,0.000906,0.000906,0.018954,0.018954
loading_percent,17.868504,17.868504,1.888156,1.888156,11.896727,11.896727,2.758555,2.758555,1.028140,1.028140,5.525884,5.525884,14.191139,14.147421,14.147421,7.579588,7.579588,4.052958,4.052958,4.857863,4.857863,1.140058,11.028759,11.0